## Task ba2f
Disclaimer - tested it multiple times and more often than not 1k iterations is not enough to converge - had to increase iterations as much as i could to fit 5 min mark - so doesn't output correct answer 100% of time

In [21]:
import numpy as np
from tqdm import tqdm # because usually needed more iteration and i needed to keep track of time

In [22]:
def read_data(fname='rosalind_ba2f.txt'):
    with open(fname, 'r') as f:
        out = [elem.strip() for elem in f.readlines()]
    k, t = [int(elem) for elem in out[0].split()]
    dna = out[1:]
    # for debug datasets (they are not /n separated)
    if ' ' in dna[0]:
        dna = dna[0].split(' ')
    return k, t, dna

def HammingDist(s1, s2):
    return sum([1 for i in range(len(s1)) if s1[i] != s2[i]])

def sample_motifs(dna, k):
    l = len(dna[0])
    t = len(dna)
    start_idxs = np.random.random_integers(0, l-k, t)
    return [dna[i][start_idxs[i]:start_idxs[i]+k] for i in range(t)]
    
def get_profile(motifs):
    k = len(motifs[0])
    t = len(motifs)
    
    pos_dict = {'A' : 0,
               'C': 1,
               'G': 2,
               'T': 3}
    prf = [[0 for j in range(k)] for elem in ['A', 'C', 'G', 'T']]
    
    for i, motif in enumerate(motifs):
        for j, elem in enumerate(motif):
            prf[pos_dict[elem]][j] += 1
                
    return np.array(prf) / t

def select_motifs(profile, dna):
    k = len(profile[0])
    l = len(dna[0])
    pos_dict = {'A' : 0,
               'C': 1,
               'G': 2,
               'T': 3}
    best_motifs = []
    
    for string in dna:
        cur_best_prob = -1
        cur_best_motif = 'idk'
        
        for i in range(l-k+1):
            cur_motif = string[i:i+k]
            prob = np.product([profile[pos_dict[elem]][j] for j, elem in enumerate(cur_motif)])
            if prob > cur_best_prob:
                cur_best_prob = prob
                cur_best_motif = cur_motif
                
        best_motifs.append(cur_best_motif)
    return best_motifs

def generate_best_motif(profile):
    k = len(profile[0])
    inv_pos_dict = {0: 'A',
               1: 'C',
               2: 'G',
               3: 'T'}
    return ''.join([inv_pos_dict[np.argmax(profile[:, i])] for i in range(k)])

def score(motifs, best_motif):
    return np.sum([HammingDist(best_motif, motif) for motif in motifs])

def randomized_motif_search(dna, k, t):
    motifs = sample_motifs(dna, k)
    best_motifs = motifs
    cur_motifs = motifs
    while 1:
        profile = get_profile(cur_motifs)
        cur_motifs = select_motifs(profile, dna)
        best_motif = generate_best_motif(profile)
        cur_score = score(cur_motifs, best_motif)
        best_score = score(best_motifs, best_motif)
        if cur_score < best_score:
            best_motifs = cur_motifs
        else:
            return best_motifs, best_score

def main(fname='rosalind_ba2f.txt', n_iter=1000):
    np.random.seed(6)
    k, t, dna = read_data(fname=fname)
    best_iter_motifs, best_iter_score = randomized_motif_search(dna, k, t)
    for _ in tqdm(range(1, n_iter)):
        cur_iter_motifs, cur_iter_score = randomized_motif_search(dna, k, t)
        
        if cur_iter_score < best_iter_score:
            best_iter_score = cur_iter_score
            best_iter_motifs = cur_iter_motifs
    for motif in best_iter_motifs:
        print(motif)

In [23]:
# this file is broken on rosalind end
read_data(fname='sample_data/ba2f/input_1.txt')

(8,
 5,
 ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA',
  'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
  'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
  'TAGATCAAGTTTCAGGTGCACGTCGGTGAACCAA',
  'TCCACCAGCTCCACGTGCAATGTTGGCCTA'])

In [24]:
main(fname='sample_data/ba2f/input_2.txt', n_iter=1500)

C:\Users\gluho\AppData\Local\Temp\ipykernel_6452\1321366715.py:17: DeprecationWarning: This function is deprecated. Please call randint(0, 34 + 1) instead
  start_idxs = np.random.random_integers(0, l-k, t)
100%|█████████████████████████████████████████████████████████████████████████████| 1499/1499 [00:05<00:00, 281.95it/s]

CGATAA
GGTTAA
GGTATA
GGTTAA
GGTTAC
GGTTAA
GGCCAA
GGTTAA


In [25]:
with open('sample_data/ba2f/output_2.txt', 'r') as f:
    out = f.readlines() 
print(out)

['CGATAA GGTTAA GGTATA GGTTAA GGTTAC GGTTAA GGCCAA GGTTAA']


In [26]:
#driver code
# main(n_iter=1500)